# TeLLMyStory
## BLUMET Thomas HALVICK Thomas LETRUNG Ethan 5A INFO Polytech Lyon - M2IA
### Notebook use on Azure AI for running the training and evaluate the model

In [1]:
# Check your python version
import sys
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [1]:
# install the transformer library from Hugging Face
%pip install transformers<4.36.2

/bin/bash: 4.36.2: No such file or directory
Note: you may need to restart the kernel to use updated packages.


In [2]:
# install other libraries use in the project
%pip install -q -U accelerate optimum 
%pip install bitsandbytes evaluate

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 52.7 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
#install library for the use of Bookcorpus and Roc-Stories
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# show the version of library
%pip show datasets

Name: datasets
Version: 3.2.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: evaluate, optimum
Note: you may need to restart the kernel to use updated packages.


In [5]:
# the crux package for the project
%pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu117/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 46.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 58.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip show auto-gptq

Name: auto_gptq
Version: 0.7.1
Summary: An easy-to-use LLMs quantization package with user-friendly apis, based on GPTQ algorithm.
Home-page: https://github.com/PanQiWei/AutoGPTQ
Author: PanQiWei
Author-email: 
License: 
Location: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages
Requires: accelerate, datasets, gekko, numpy, peft, rouge, safetensors, sentencepiece, torch, tqdm, transformers
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
# install the torch library for using torch version with CUDA
#%pip uninstall torch -y

Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install torch --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip show torch

Name: torch
Version: 2.5.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, auto_gptq, bitsandbytes, optimum, peft
Note: you may need to restart the kernel to use updated packages.


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig
import torch

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
print(f"Is CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(torch.cuda.current_device())}")

Is CUDA available: True
CUDA device: Tesla T4


In [11]:
import optimum
import auto_gptq

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.


In [12]:
from transformers.utils import is_auto_gptq_available, is_optimum_available
print(is_auto_gptq_available())
print(is_optimum_available())

True
True


In [13]:
device = torch.device("cuda:0")

In [14]:
# The model name is the name of the model you want to use, you can use the model from HuggingFace or the model you trained
# The model is a pretrained model from the Hugging space created by TheBloke (cf https://huggingface.co/TheBloke)
# Further infos for GPTQ model availbale here : https://huggingface.co/docs/transformers/quantization/gptq

model_name = "TheBloke/zephyr-7B-beta-GPTQ" 

tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True,padding_side="left")
#if training disable_exllama=True
quantization_config_loading = GPTQConfig(
                                bits=4,
                                group_size=128,
                                disable_exllama=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config_loading, device_map="auto")
model = model.to(device)
print(model)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/quantizers/auto.py:195: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwr

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-05)
    (rotary_emb): MistralRotaryEmbedding()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)


In [15]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.GPTQ: 'gptq'>,
 'bits': 4,
 'tokenizer': None,
 'dataset': None,
 'group_size': 128,
 'damp_percent': 0.1,
 'desc_act': True,
 'sym': True,
 'true_sequential': True,
 'use_cuda_fp16': False,
 'model_seqlen': 4096,
 'block_name_to_quantize': 'model.layers',
 'module_name_preceding_first_block': ['model.embed_tokens'],
 'batch_size': 1,
 'pad_token_id': None,
 'use_exllama': False,
 'max_input_length': None,
 'exllama_config': {'version': <ExllamaVersion.ONE: 1>},
 'cache_block_outputs': True,
 'modules_in_block_to_quantize': None}

# Testing the pretrained model

In [16]:
def generate_text(input_text,max_new_tokens=512,top_k=50,top_p=0.95,temperature=0.7,no_grad=False):
    tokenizer.pad_token_id = tokenizer.eos_token_id
    input_ids = tokenizer.encode(input_text, padding=True, return_tensors="pt").to(device)
    attention_mask = input_ids.ne(tokenizer.pad_token_id).long().to(device)
    output = None
    if no_grad:
        with torch.no_grad():
            output = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens, top_k=top_k, top_p=top_p, temperature=temperature,do_sample=True)
    else:
        output = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens, top_k=top_k, top_p=top_p, temperature=temperature,do_sample=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [17]:
#so test by generating a story with the following prompt
prompt = "Once upon a time an ice-cream met a spoon and they fell in love"
prompt2 = "Hello my name is"

In [17]:
#overview of the story
import time

print(f'Prompt : {prompt} \n')
start = time.time()
print(f'Story : {generate_text(prompt,max_new_tokens=100,temperature=1)} \n')
end = time.time()
print(f'Time to generate the story : {end-start} s')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt : Once upon a time an ice-cream met a spoon and they fell in love 

Story : Once upon a time an ice-cream met a spoon and they fell in love... This is their fairytale-like love story, set in a magical garden full of lollies and sprinkles and ice-creams, and told with the help of beautiful illustrations. And in the end, it will make you hungry for ice-cream!
This is the second book by Tania CC in the Tales series. Like all the Tales stories, it is beautifully illustrated by Ana Kovačić and is a fairytale in the true 

Time to generate the story : 97.36324572563171 s


In [18]:
print(f'{generate_text(prompt2,max_new_tokens=100,temperature=0.95)} \n')

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Hello my name is Katie and I am a 20 year old girl from the UK. I have been a member of this site for a few months now and I have been reading a lot of the posts and I have noticed that there are a lot of people who are struggling with their mental health and I wanted to share my story with you all.

I have suffered from anxiety and depression for as long as I can remember. I remember being a child and feeling like I didn't fit in with


## If you want to see the use of the current hardware (GPU, CPU)

In [17]:
#hardware monitoring
%pip install psutil gputil
import psutil
import GPUtil
from tabulate import tabulate
import time
def print_gpu_cpu_usage(interval=1):
    while True:
        # Obtenir l'utilisation du CPU
        cpu_percent = psutil.cpu_percent(interval=0.1)

        # Obtenir l'utilisation de la mémoire
        memory_info = psutil.virtual_memory()
        memory_percent = memory_info.percent

        # Obtenir l'utilisation du GPU
        gpus = GPUtil.getGPUs()
        list_gpus = []
        for gpu in gpus:
            list_gpus.append((
                gpu.id,
                f"{gpu.name}",
                f"{gpu.load*100}%",
                f"{gpu.memoryFree}MB",
                f"{gpu.memoryUsed}MB",
                f"{gpu.memoryTotal}MB",
                f"{gpu.temperature} C"
            ))

        # Afficher les informations de manière tabulée
        print(f"CPU Utilization: {cpu_percent}%")
        print(f"Memory Utilization: {memory_percent}%")
        if list_gpus:
            print(tabulate(list_gpus, headers=("ID", "Name", "Load", "Free Memory", "Used Memory", "Total Memory", "Temperature")))
        else:
            print("No GPU detected.")
        
        time.sleep(interval)

print_gpu_cpu_usage()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=a58fd3059df40b0e4982a455fac23df7c4e6764b950c4c5d1d72f1095e4c7ece
  Stored in directory: /home/azureuser/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil
Note: you may need to restart the kernel to use updated packages.
CPU Utilization: 0.0%
Memory Utilization: 21.1%
  ID  Name      Load    Free Memory    Used Memory    Total Memory    Temperature
----  --------  ------  -------------  -------------  --------------  -------------
   0  Tesla T4  0.0%    11055.0MB      4893.0MB       16384.0MB       36.0 C
CPU Utilization: 0.0%
Memory Utilization: 21.1%
  ID  Name      Load    Free Memory    Used Memory    Total Memory    Temperature
----  --------  ------  -------------  -------------  --------------  -------------
   0  Tesla T4  0.0%    11055.0MB      4893.0MB       16384.0MB       36.0 C
CPU Utilization: 2.

KeyboardInterrupt: 

In [29]:
#clean up GPU memory if GPU out of memory
#torch.cuda.empty_cache()

#show the ressource usage
#!nvidia-smi

#if other problem, restart the kernel or try the following command
#Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false) to enable or disable the parallelism in the tokenizers library.
#import os
#os.environ["TOKENIZERS_PARALLELISM"] = "true"
#model.config.quantization_config.to_dict()

#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
#os.getenv('PYTORCH_CUDA_ALLOC_CONF')

Thu Jan 23 22:24:06 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.216.03             Driver Version: 535.216.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000001:00:00.0 Off |                  Off |
| N/A   38C    P0              26W /  70W |  15615MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Training part with dataset loading

In [18]:
from datasets import load_dataset

In [23]:
#loading the bookcorpus dataset
bookcorpus = load_dataset("bookcorpus",trust_remote_code=True) 


Generating train split: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74004228/74004228 [17:05<00:00, 72179.97 examples/s]


In [25]:
bookcorpus

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 74004228
    })
})

In [19]:
#loading the rocstories dataset
roc = load_dataset("wza/roc_stories",trust_remote_code=True)

Generating train split: 98161 examples [00:10, 9736.79 examples/s] 


In [20]:
roc

DatasetDict({
    train: Dataset({
        features: ['storyid', 'storytitle', 'sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5'],
        num_rows: 98161
    })
})

In [26]:
train_dataset = bookcorpus["train"]
print(f'Example of sentence in train_dataset= ',train_dataset[1])

Example of sentence in train_dataset=  {'text': 'but just one look at a minion sent him practically catatonic .'}


In [21]:
train_dataset = roc["train"]
print(f'Example of sentence in train_dataset= ',train_dataset[1])

Example of sentence in train_dataset=  {'storyid': '617e7ada-3878-488d-bd56-40695b91f053', 'storytitle': 'The Bike Accident', 'sentence1': 'Carrie had just learned how to ride a bike.', 'sentence2': "She didn't have a bike of her own.", 'sentence3': "Carrie would sneak rides on her sister's bike.", 'sentence4': 'She got nervous on a hill and crashed into a wall.', 'sentence5': 'The bike frame bent and Carrie got a deep gash on her leg.'}


In [25]:
from transformers import TrainingArguments, Trainer, TrainerCallback
from evaluate import load

In [29]:
#evaluate the model with rouge score
%pip install rouge_score nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 33.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=aee4cf37725a66b7d2b4a2077ebb176d9ba01e50e2a51c19c810f2aa1d6e4e1a
  Stored in directory: /home/azureuser/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [33]:
# Preprocess the dataset with the tokenizer before training
def tokenize_bc_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

def tokenize_roc_function(examples):
    return tokenizer(examples["sentence1"],examples["sentence2"],examples["sentence3"],examples["sentence4"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = roc.map(tokenize_roc_function, batched=True) 
#tokenized_datasets1 = bookcorpus.map(tokenize_bc_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(['storyid', 'storytitle', 'sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5'])
#tokenized_datasets1 = tokenized_datasets1.remove_columns(["text"])

tokenized_datasets.set_format("torch")
#tokenized_datasets1.set_format("torch")

Map:   0%|          | 0/98161 [00:00<?, ? examples/s]

Map: 100%|██████████| 98161/98161 [00:34<00:00, 2843.13 examples/s]


In [43]:
#split the tokenized_datasets mapping with dataset into training and validation (90% training and 10% validation)
tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.2)
tokenized_datasets["validation"]=tokenized_datasets["test"]

In [44]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 63607
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15902
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15902
    })
})

In [30]:
# metrics lodaded from the evaluate library
rouge = load("rouge")
meteor = load("meteor")
bleu = load("bleu")
accuracy = load("accuracy")

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/azureuser/nltk_data...


In [35]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    accuracy_result = accuracy.compute(predictions=decoded_preds, references=decoded_labels)
    
    return {
        "rouge": rouge_result,
        "meteor": meteor_result,
        "bleu": bleu_result,
        "accuracy": accuracy_result,
    }

In [ ]:
%pip install peft

### [Some docs about peft fine-tuning methods](https://huggingface.co/docs/transformers/peft)

In [55]:
# Remarks: some error in the code, the model is not trained
#for training the model the GPTQ model requires some configuration using peft
from peft import PeftModel, PeftConfig

peft_config = PeftConfig(
    task_type="CAUSAL_LM",
    peft_type="adapter",
    auto_mapping=True,
    base_model_name_or_path=model_name,
)
model_with_adapters = PeftModel(model, peft_config)
#or if use directly the model with adaptaters
#model.add_adapter(peft_config)
#enable or disable adaptaters
#model.enable_adapters()
#model.disable_adapters()

# Config for the training with the Trainer API
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    save_steps=500,
    eval_steps=500,
)

# Callback to print the metrics during training
class LogMetricsCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is not None:
            print(f"Metrics at step {state.global_step}: {metrics}")
            
    def on_train_begin(self, args, state, control, **kwargs):
        print("Training is starting")

    def on_train_end(self, args, state, control, **kwargs):
        print("Training is finished")

    def on_log(self, args, state, control, logs=None, **kwargs):
        print(f"Step: {state.global_step}, Loss: {logs['loss']}")


trainer = Trainer(
    model=model_with_adapters,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    callbacks=[LogMetricsCallback()],
)

trainer.train()

#save the model
save_dir = "roc_trained_model"
model_with_adapters.save_pretrained(save_dir)

KeyError: 'adapter'

# Model evaluation with metrics
### [Cf this link to Hugging Face about evaluation model](https://huggingface.co/evaluate-metric)

In [56]:
from evaluate import load
#model.eval()

# Setup evaluation 
metric = load("rouge")
metric2 = load("meteor")
# metric3 = load("accuracy")
# metric4 = load("bleu")
# metric5 = load("f1")
# metric6 = load("precision")
# metric7 = load("recall")


def evaluate_model(prompts, references,max_tokens=100,metric=metric):
    generated_texts = [generate_text(prompt,max_new_tokens=max_tokens) for prompt in prompts]
    print(f'Prediction= {[generated_text for generated_text in generated_texts]}\n')
    results = metric.compute(predictions=generated_texts, references=references)
    return results

def rouge_metric(generated_text, reference):
    return metric.compute(predictions=[generated_text], references=[reference])

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [57]:
#evaluation of the pretrained model without additional training
prompts = ["The neverending day began with a beautiful sunshine and an AI robot which was seeking humans on the desert Earth."]
references = ["As the sun rose on the barren wasteland that once was Earth, an AI robot named X-979 set out on its daily mission. The robot's programming had been designed to search for signs of human life, a task that had proved futile for many years."]

print(f'Prompts= {prompts[0]}\n')
results = evaluate_model(prompts, references,metric=metric2)
print(f'References= {[reference for reference in references]}\n')
print(f'Meteor_score = {results}')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompts= The neverending day began with a beautiful sunshine and an AI robot which was seeking humans on the desert Earth.

Prediction= ['The neverending day began with a beautiful sunshine and an AI robot which was seeking humans on the desert Earth. Its mission was simple: to locate the remaining humans and take them to a safe place. The AI robot was programmed to avoid the hazardous areas that were filled with mutated creatures, but it encountered a group of humans who were fighting against the mutated creatures.\n\nThe AI robot immediately identified that the group was a potential threat to its mission, so it tried to communicate with them. The group, however, was hesitant and refused to cooperate with the robot. The']

References= ["As the sun rose on the barren wasteland that once was Earth, an AI robot named X-979 set out on its daily mission. The robot's programming had been designed to search for signs of human life, a task that had proved futile for many years."]

Rouge_score

Prompts= The neverending day began with a beautiful sunshine and an AI robot which was seeking humans on the desert Earth.

Prediction= ['The neverending day began with a beautiful sunshine and an AI robot which was seeking humans on the desert Earth. Its mission was simple: to locate the remaining humans and take them to a safe place. The AI robot was programmed to avoid the hazardous areas that were filled with mutated creatures, but it encountered a group of humans who were fighting against the mutated creatures.\n\nThe AI robot immediately identified that the group was a potential threat to its mission, so it tried to communicate with them. The group, however, was hesitant and refused to cooperate with the robot. The']

References= ["As the sun rose on the barren wasteland that once was Earth, an AI robot named X-979 set out on its daily mission. The robot's programming had been designed to search for signs of human life, a task that had proved futile for many years."]

Meteor_score = {'meteor': 0.2721283422459893}


In [42]:
prompts = ["The neverending day began with a beautiful sunshine and an AI robot which was seeking humans on the desert Earth."]
references = ["As the sun rose on the barren wasteland that once was Earth, an AI robot named X-979 set out on its daily mission. The robot's programming had been designed to search for signs of human life, a task that had proved futile for many years."]
#prompts = ["Once upon a time an ice-cream met a spoon and they fell in love"]
#references = ["Once upon a time, there was an ice-cream named Charlie who was scooped from the freezer and placed in a cup. Charlie was a friendly and outgoing ice-cream who loved making friends. As soon as Charlie saw a spoon named Sarah, he knew he had found his soulmate."]

print(f'Prompts= {prompts[0]}\n')
results = evaluate_model(prompts, references)
print(f'References= {[reference for reference in references]}\n')
print(f'Rouge_score = {results}')



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompts= The neverending day began with a beautiful sunshine and an AI robot which was seeking humans on the desert Earth.

Prediction= ["The neverending day began with a beautiful sunshine and an AI robot which was seeking humans on the desert Earth. I was the only human that I have found, and the AI robot was happy to find me.\n\nAfter that, I became a part of his journey. It was hard to say how long we have been traveling together. It might be a few days, weeks, or even months. Time had lost its meaning on the barren and lifeless planet.\n\nThe AI robot's name was X-113, and it had been designed by humans to be a part of"]

References= ["As the sun rose on the barren wasteland that once was Earth, an AI robot named X-979 set out on its daily mission. The robot's programming had been designed to search for signs of human life, a task that had proved futile for many years."]

Rouge_score = {'rouge1': 0.3401360544217687, 'rouge2': 0.09655172413793103, 'rougeL': 0.2040816326530612, 'rou

Prompts= The neverending day began with a beautiful sunshine and an AI robot which was seeking humans on the desert Earth.

Prediction= ["The neverending day began with a beautiful sunshine and an AI robot which was seeking humans on the desert Earth. I was the only human that I have found, and the AI robot was happy to find me.\n\nAfter that, I became a part of his journey. It was hard to say how long we have been traveling together. It might be a few days, weeks, or even months. Time had lost its meaning on the barren and lifeless planet.\n\nThe AI robot's name was X-113, and it had been designed by humans to be a part of"]

References= ["As the sun rose on the barren wasteland that once was Earth, an AI robot named X-979 set out on its daily mission. The robot's programming had been designed to search for signs of human life, a task that had proved futile for many years."]

Rouge_score = {'rouge1': 0.3401360544217687, 'rouge2': 0.09655172413793103, 'rougeL': 0.2040816326530612, 'rougeLsum': 0.2448979591836734}


# Display the Gradio app in the notebook

In [19]:
%pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 48.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 60.6 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [34]:
import gradio as gr
import time
from tqdm import tqdm 
import matplotlib.pyplot as plt
#import threading  

time_story = 0

def generate_response(input,history: list[tuple[str, str]],max_tokens, temperature, top_p):
    messages=[]
    for val in history:
        # Directly access content using "content" key
        messages.extend([{"role": "user", "content": val.get("content")}, {"role": "assistant", "content": val.get("content")}]) if val else None

    messages.append({"role": "user", "content": input})
    start = time.time()
    
    output = generate_text(input,max_new_tokens=max_tokens, top_p=top_p, temperature=temperature)
    end = time.time()
    time_story= end-start
    print(f'Time to generate the story : {time_story} s')

    history.append((input,output))
    print(f'max_tokens = {max_tokens}')
    print(f'temperature = {temperature}')
    print(f'top_p = {top_p}')
    yield output

#define the chatinterface
title = "TeLLMyStory"
description = "A LLM for stories generation aiming the reinforcement of the controllability aspect"
theme = gr.Theme.from_hub("Yntec/HaleyCH_Theme_Yellow_Blue")
examples=[["Once upon a time a witch named Malefique was against the wedding of her daughter with the son of the king of the nearby kingdom."],
        ["Once upon a time an ice-cream met a spoon and they fell in love"],
        ["The neverending day began with a beautiful sunshine and an AI robot which was seeking humans on the desert Earth."]]

demo = gr.ChatInterface(
      generate_response,
      type="messages",
      title=title,
      description=description,
      theme=theme,
      examples=examples,
      additional_inputs=[
          gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="Max new tokens"),
          gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="Temperature"),
          gr.Slider(
              minimum=0.1,
              maximum=1.0,
              value=0.95,
              step=0.05,
              label="Top-p (nucleus sampling)",
          ),
      ],
    
    stop_btn="Stop",
    delete_cache=[60,60],
    show_progress="full",
    save_history=True,
  )

#close the previous gradio app window in the notebook (else windows are added on the output cell)
demo.close()

if __name__ == "__main__":
    demo.launch(share=True,debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://456f4509e5a857e701.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Time to generate the story : 46.24886965751648 s
max_tokens = 50
temperature = 0.7
top_p = 0.95
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://456f4509e5a857e701.gradio.live
